In [ ]:
import torch 
from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.load_state_dict(torch.load('../models/checkpoints/bert_20.07_13:15.pth', map_location='cpu'))
model.eval()  

In [13]:
inputs = tokenizer(
    'heyy',
    'hello',
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)
inputs = {k: v.to('cpu') for k, v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = logits.softmax(1)

In [19]:
inputs

{'input_ids': tensor([[ 101, 4931, 2100,  102, 7592,  102,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      

In [24]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1155, -0.2844]]), hidden_states=None, attentions=None)

In [25]:
batch_size = 1
max_length = 128  # или другая длина, которую вы используете

dummy_input = {
    'input_ids': torch.randint(0, 1000, (batch_size, max_length), dtype=torch.long),
    'token_type_ids': torch.zeros(batch_size, max_length, dtype=torch.long),
    'attention_mask': torch.ones(batch_size, max_length, dtype=torch.long)
}

# Конвертация в ONNX
torch.onnx.export(
    model,                          # модель для экспорта
    (dummy_input['input_ids'], 
     dummy_input['attention_mask'], 
     dummy_input['token_type_ids']), # входные данные (в виде tuple)
    "bert_model.onnx",             # имя выходного файла
    export_params=True,            # сохранить веса модели
    opset_version=14,              # версия ONNX opset
    do_constant_folding=True,      # оптимизация констант
    input_names=['input_ids', 'attention_mask', 'token_type_ids'],  # имена входов
    output_names=['logits'],           # имена выходов
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence'},
        'attention_mask': {0: 'batch_size', 1: 'sequence'},
        'token_type_ids': {0: 'batch_size', 1: 'sequence'},
        'last_hidden_state': {0: 'batch_size', 1: 'sequence'},
        'pooler_output': {0: 'batch_size'}
    }
)

/home/dmitrij/Документы/envs/base1/lib/python3.12/site-packages/torch/onnx/utils.py:2135: UserWarning: Provided key last_hidden_state for dynamic axes is not a valid input/output name
  warnings.warn(
/home/dmitrij/Документы/envs/base1/lib/python3.12/site-packages/torch/onnx/utils.py:2135: UserWarning: Provided key pooler_output for dynamic axes is not a valid input/output name
  warnings.warn(


In [ ]:

# Экспорт модели в формат ONNX
dummy_input = torch.randn(1, 128, device='cpu')  # Пример входного тензора
onnx_file_path = "model.onnx"  # Путь для сохранения ONNX файла

# Экспорт модели
with torch.no_grad():
    torch.onnx.export(
        model,  # Модель PyTorch
        dummy_input,  # Пример входных данных
        onnx_file_path,  # Путь для сохранения ONNX файла
        export_params=True,  # Экспортировать обученные параметры
        opset_version=14,  # Версия ONNX операторов
        do_constant_folding=True,  # Оптимизация констант
        input_names=["input"],  # Имя входного тензора
        output_names=["output"],  # Имя выходного тензора
        dynamic_axes={
            "input": {0: "batch_size"},  # Динамический размер батча
            "output": {0: "batch_size"},
        },
    )

print(f"Модель успешно экспортирована в файл: {onnx_file_path}")  